# Cerebellar statistical overlay

This demo shows the [SUIT atlas](https://www.diedrichsenlab.org/imaging/suit.htm). This jupyter notebook mimics the [SUIT atlas web page](https://niivue.com/demos/features/mesh.atlas.suit.html).

**Note atlas_values and outline_border will require upgrade to NiiVue 0.65**

In [ ]:
import ipywidgets as widgets
from ipyniivue import NiiVue, ShowRender, SliceType

nv = NiiVue(
    back_color=(1, 1, 1, 1),
    show_3d_crosshair=True,
    is_colorbar=True,
)

kCurvLayer = 0
kAltasLayer = 1
kStatLayer = 2

mesh_layers = [
    {"path": "./images/SUIT.shape.gii", "opacity": 1.0, "colormap": "gray", "cal_min": -1.5, "cal_max": 1   },
    {"path": "./images/Lobules.label.gii", "opacity": 0.05 },
    {"path": "./images/Lobules.label.gii", "opacity": 0.05 }
]

nv.load_meshes([
        {"path": "./images/FLAT.surf.gii", "layers": mesh_layers },
     ])

nv.opts.show_legend = False
nv.meshes[0].layers[0].colorbar_visible = False
nv.set_render_azimuth_elevation(0,90)
nv.set_mesh_shader(nv.meshes[0].id, "Rim")

@nv.on_mesh_loaded
def on_mesh_loaded(mesh):
    nv.meshes[0].layers[kStatLayer].cal_min = 2.3
    nv.meshes[0].layers[kStatLayer].cal_max = 5
    nv.meshes[0].layers[kStatLayer].colormap = 'warm'
    nv.meshes[0].layers[kStatLayer].colormap_negative = 'winter'
    # nv.meshes[0].layers[kStatLayer].atlas_values = [[0, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 6, 0, -4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
    nv.meshes[0].layers[kStatLayer].atlas_values = [0, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 6, 0, -4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    nv.set_mesh_shader(nv.meshes[0].id, "Rim")
    nv.meshes[0].layers[kCurvLayer].colorbar_visible = False
    nv.meshes[0].layers[kAltasLayer].colorbar_visible = False

## User interface widgets

curv_slider = widgets.IntSlider(
    value=50,
    min=1,
    max=99,
    description="Curvature",
    readout=False
)

def on_curv_change(change):
    """Set curve transparency."""
    nv.set_mesh_layer_property(
        mesh_id=nv.meshes[0].id,
        layer_index=kCurvLayer,
        attribute="opacity",
        value=change["new"] * 0.01,
    )

curv_slider.observe(on_curv_change, names="value")

atlas_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=99,
    description="Atlas",
    readout=False
)

def on_atlas_change(change):
    """Set atlas transparency."""
    nv.set_mesh_layer_property(
        mesh_id=nv.meshes[0].id,
        layer_index=kAltasLayer,
        attribute="opacity",
        value=change["new"] * 0.01,
    )

atlas_slider.observe(on_atlas_change, names="value")

stat_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=99,
    description="Stats",
    readout=False
)

def on_stat_change(change):
    """Set stat transparency."""
    nv.set_mesh_layer_property(
        mesh_id=nv.meshes[0].id,
        layer_index=kStatLayer,
        attribute="opacity",
        value=change["new"] * 0.01,
    )

stat_slider.observe(on_stat_change, names="value")

border_options = ["Dark border", "Transparent border", "No border", "Opaque border"]
border_dropdown = widgets.Dropdown(
    options=border_options,
    value="Opaque border",
    description="Border",
)

def on_border_change(change):
    """Set mesh border style."""
    value_name = change["new"]

    # Default borderValue for "No border"
    borderValue = 0.0
    if value_name == "Dark border":
        borderValue = -0.01  # MRIcroGL convention: negative = dark border
    elif value_name == "Transparent border":
        borderValue = 0.01   # small positive = transparent border
    elif value_name == "Opaque border":
        borderValue = 1.0    # fully opaque border
    # else "No border" → 0.0
    nv.set_mesh_layer_property(nv.meshes[0].id, kAltasLayer, "outline_border", borderValue)

border_dropdown.observe(on_border_change, names="value")

## Display

widgets.VBox(
    [
        widgets.HBox([curv_slider, atlas_slider, stat_slider, border_dropdown]),
        nv,
    ]
)
